## Using Santander Cycling data to find popular destinations for bike trips 
https://cycling.data.tfl.gov.uk/

In [1]:
import matplotlib.pyplot as plt
import pandas as pd

%matplotlib inline

import json
import requests
from xml.etree import ElementTree as ET
#from io import BytesIO
#from io import StringIO

import urllib.request
from urllib.request import urlopen
import os

In [2]:
# Get the path to the current Notebook file
notebook_path = %pwd

# import bike trips data
relative_file_path = "biketrip2022"
folder_path = notebook_path + '/' + relative_file_path

# Initialise an empty DataFrame to hold the merged data
merged_df = pd.DataFrame()

# Iterate over files in a folder
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        
        # Read CSV files
        df = pd.read_csv(file_path)
        
        # Merge data into total DataFrame
        merged_df = pd.concat([merged_df, df], ignore_index=True)

# Save the merged data as a new CSV file
## merged_csv_path = "bike_data.csv"
## merged_df.to_csv(merged_csv_path, index=False)
## print("merge finish and saved as:", merged_csv_path)

In [3]:
bike_data = merged_df
bike_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8876811 entries, 0 to 8876810
Data columns (total 9 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Rental Id          int64  
 1   Duration           int64  
 2   Bike Id            int64  
 3   End Date           object 
 4   EndStation Id      float64
 5   EndStation Name    object 
 6   Start Date         object 
 7   StartStation Id    float64
 8   StartStation Name  object 
dtypes: float64(2), int64(3), object(4)
memory usage: 609.5+ MB


In [4]:
# List of columns to keep (exclude the ones you want to drop)
columns_to_keep = ['StartStation Id','EndStation Id']

# Read the CSV file with selected columns only
#####bike_data_filtered=pd.read_csv("bike_data.csv", usecols=columns_to_keep)
bike_data_filtered = bike_data[columns_to_keep]

In [5]:
# Define the output CSV file path
#output_csv_path = 'bike_data_filtered1.csv'

# Export the DataFrame to a CSV file
#bike_data.to_csv(output_csv_path, index=False)

#print(f"DataFrame exported to {output_csv_path}")

In [6]:
#url containing the locations of all Santander bike stops
bike_locations = "https://tfl.gov.uk/tfl/syndication/feeds/cycle-hire/livecyclehireupdates.xml"

def get_location_from_xml(url):
    response = requests.get(url)
    root = ET.fromstring(response.content)
    
    col_list = []
    for i in range(0, len(root)):
        id_list = int(root[i][0].text)
        name_list = str(root[i][1].text)
        lat_list = float(root[i][3].text)
        lon_list = float(root[i][4].text)
        capacity_list = int(root[i][12].text)
        col_list.append([id_list,name_list,lat_list,lon_list,capacity_list])
    
    all_locs = pd.DataFrame(list(col_list), columns = ["id","name","lat","lon","capacity"])
#    all_locs.to_dataframe(url.split('/')[7].split('.')[0]+'.csv', header=True, index=None)
    print("Shape of cycle hire locations: ",all_locs.shape)
    
    return all_locs

bike_locations = get_location_from_xml(bike_locations)

Shape of cycle hire locations:  (795, 5)


In [7]:
# Define the output CSV file path
##output_csv_path = 'bike_locations.csv'

# Export the DataFrame to a CSV file
##bike_locations.to_csv(output_csv_path, index=False)

##print(f"DataFrame exported to {output_csv_path}")

In [8]:
bike_locations

,id,name,lat,lon,capacity
0,1,"River Street , Clerkenwell",51.529163,-0.109971,0
1,2,"Phillimore Gardens, Kensington",51.499607,-0.197574,1
2,3,"Christopher Street, Liverpool Street",51.521284,-0.084606,0
3,4,"St. Chad's Street, King's Cross",51.530059,-0.120974,0
4,5,"Sedding Street, Sloane Square",51.493130,-0.156876,1
...,...,...,...,...,...
790,847,"South Bermondsey Station, Bermondsey",51.488067,-0.057775,0
791,849,"Crimscott Street, Bermondsey",51.495598,-0.078893,0
792,850,"Brandon Street, Walworth",51.489102,-0.091549,1
793,851,"The Blue, Bermondsey",51.492221,-0.062513,1


In [9]:
bike_data=bike_data_filtered

# Group by 'EndStation Id' and count the occurrences
inbound_num = bike_data.groupby('EndStation Id').size().reset_index(name='inbound_counts')

# Merge the counts with the original DataFrame
bike_data_sum = bike_data.merge(inbound_num, on='EndStation Id')


# Keep only 'EndStation Id' and 'inbound_num' columns
destination_data_sum = bike_data_sum[['EndStation Id', 'inbound_counts']].drop_duplicates()

# Print the result
destination_data_sum

,EndStation Id,inbound_counts
0,310.0,10377
10377,11.0,11315
21692,70.0,4057
25749,804.0,18184
43933,14.0,39494
...,...,...
8551658,752.0,10482
8562140,147.0,467
8562607,65.0,985
8563592,846.0,810


In [10]:
# Merge bike_locations with destination_sum based on 'EndStation Id'
destination_dataXY = destination_data_sum.merge(bike_locations, left_on='EndStation Id', right_on='id', how='inner')

# Print the merged DataFrame
destination_dataXY

,EndStation Id,inbound_counts,id,name,lat,lon,capacity
0,310.0,10377,310,"Black Prince Road, Vauxhall",51.490868,-0.116912,1
1,11.0,11315,11,"Brunswick Square, Bloomsbury",51.523951,-0.122502,0
2,70.0,4057,70,"Calshot Street , King's Cross",51.531361,-0.117070,0
3,804.0,18184,804,"Good's Way, King's Cross",51.534668,-0.125079,2
4,14.0,39494,14,"Argyle Street, Kings Cross",51.529416,-0.123944,1
...,...,...,...,...,...,...,...
783,752.0,10482,752,"London Street, Paddington",51.515117,-0.173713,0
784,147.0,467,147,"Portugal Street, Holborn",51.514943,-0.116146,1
785,65.0,985,65,"Gower Place , Euston",51.525228,-0.135189,1
786,846.0,810,846,"Burgess Park Albany Road, Walworth",51.482240,-0.094284,0


In [11]:
# Define the output CSV file path
output_csv_path = 'destination_dataXY.csv'

# Export the DataFrame to a CSV file
destination_dataXY.to_csv(output_csv_path, index=False)

print(f"DataFrame exported to {output_csv_path}")